# Starter Code

In [2]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report
from joblib import dump, load
import math

df_2018 = pd.read_csv('data/2018.csv')
df_2017 = pd.read_csv('data/2017.csv')
df_2016 = pd.read_csv('data/2016.csv')

df_2018

,posteam,yardline_100,game_date,game_seconds_remaining,down,ydstogo,score_differential,fourth_down_converted,fourth_down_failed,field_goal_attempt,punt_attempt
0,ATL,1.0,2018-09-06,3355.0,4.0,1.0,0.0,0.0,1.0,0.0,0.0
1,PHI,92.0,2018-09-06,3260.0,4.0,4.0,0.0,0.0,0.0,0.0,1.0
2,ATL,3.0,2018-09-06,3003.0,4.0,3.0,0.0,0.0,0.0,1.0,0.0
3,PHI,65.0,2018-09-06,2733.0,4.0,18.0,-3.0,0.0,0.0,0.0,1.0
4,ATL,89.0,2018-09-06,2658.0,4.0,19.0,3.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
3502,NO,70.0,2018-12-17,1483.0,4.0,4.0,-1.0,0.0,0.0,0.0,0.0
3503,NO,53.0,2018-12-17,1363.0,4.0,8.0,-1.0,0.0,0.0,0.0,1.0
3504,CAR,58.0,2018-12-17,536.0,4.0,16.0,-5.0,0.0,0.0,0.0,1.0
3505,NO,14.0,2018-12-17,150.0,4.0,1.0,5.0,1.0,0.0,0.0,0.0


In [3]:
df_2018 = pd.read_csv('data/2018.csv')
df_2017 = pd.read_csv('data/2017.csv')
df_2016 = pd.read_csv('data/2016.csv')

def add_classifier(df):
    conditions = [
        (df['fourth_down_converted'] == 1.0),
        (df['fourth_down_failed'] == 1.0),
        (df['field_goal_attempt'] == 1.0),
        (df['punt_attempt'] == 1.0)
    ]

    results = [0, 1, 2, 3]

    df['class'] = np.select(conditions, results)
    return df

def get_classifier_convert(df):
    conditions = [
        (df['fourth_down_converted'] == 1.0),
        (df['fourth_down_failed'] == 1.0),
        (df['field_goal_attempt'] == 1.0),
        (df['punt_attempt'] == 1.0)
    ]

    results = ['CONVERTED', 'FAILED', 'FIELD_GOAL', 'PUNT']

    y = np.select(conditions, results)
    return y

def get_classifier_attempt(df):
    conditions = [
        (df['fourth_down_converted'] == 1.0) | (df['fourth_down_failed'] == 1.0),
        (df['field_goal_attempt'] == 1.0),
        (df['punt_attempt'] == 1.0)
    ]

    results = ['ATTEMPTED', 'FIELD_GOAL', 'PUNT']

    y = np.select(conditions, results)
    return y

def ready_data_convert(df):
    df = df.dropna()
    y = get_classifier_convert(df)
    df = df.drop(columns=['posteam', 'fourth_down_converted', 'fourth_down_failed', 'field_goal_attempt', 'punt_attempt', 'game_date', 'down'])
    return df, y

def ready_data_attempt(df):
    df = df.dropna()
    y = get_classifier_attempt(df)
    df = df.drop(columns=['posteam', 'fourth_down_converted', 'fourth_down_failed', 'field_goal_attempt', 'punt_attempt', 'game_date', 'down'])
    return df, y

# KNN

# Convert 4th Down

# Default Params

In [4]:
x, y = ready_data_convert(pd.concat([df_2016, df_2017, df_2018]))
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
print(f"Test Accuracy: {knn.score(X_test, y_test)}")
print(f"Train Accuracy: {knn.score(X_train, y_train)}")

report = classification_report(y_test, knn.predict(X_test), zero_division=0)
print(report)

C:\Users\Jcsst\ana\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\Jcsst\ana\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True o

Test Accuracy: 0.8118854211201368
Train Accuracy: 0.8545007483429549
              precision    recall  f1-score   support

           0       0.07      0.02      0.04        85
   CONVERTED       0.34      0.20      0.25       160
      FAILED       0.37      0.27      0.31       154
  FIELD_GOAL       0.76      0.91      0.83       542
        PUNT       0.91      0.95      0.93      1398

    accuracy                           0.81      2339
   macro avg       0.49      0.47      0.47      2339
weighted avg       0.77      0.81      0.79      2339



C:\Users\Jcsst\ana\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


# Cross Validation

In [6]:
x, y = ready_data_convert(pd.concat([df_2016, df_2017, df_2018]))
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)

parameters = {
    'n_neighbors': range(1, 20),
}
knn = RandomizedSearchCV(KNeighborsClassifier(), parameters, random_state=0, n_iter=200)

knn.fit(X_train, y_train)

#dump(knn.best_estimator_, '../models/random_forest/convert_cv.joblib')

print(knn.best_params_)
print(f"Test Accuracy: {knn.score(X_test, y_test)}")
print(f"Train Accuracy: {knn.score(X_train, y_train)}")

report = classification_report(y_test, knn.predict(X_test), zero_division=0)
print(report)

C:\Users\Jcsst\ana\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 19 is smaller than n_iter=200. Running 19 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
C:\Users\Jcsst\ana\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\Jcsst\ana\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts alon

{'n_neighbors': 12}
Test Accuracy: 0.8182984181274049


C:\Users\Jcsst\ana\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Train Accuracy: 0.8362197990164636


C:\Users\Jcsst\ana\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


              precision    recall  f1-score   support

           0       0.12      0.01      0.02        85
   CONVERTED       0.43      0.10      0.16       160
      FAILED       0.41      0.21      0.28       154
  FIELD_GOAL       0.75      0.92      0.83       542
        PUNT       0.88      0.97      0.93      1398

    accuracy                           0.82      2339
   macro avg       0.52      0.45      0.44      2339
weighted avg       0.76      0.82      0.78      2339



# Attempt

# Default Params

In [45]:
x, y = ready_data_attempt(pd.concat([df_2016, df_2017, df_2018]))
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
print(f"Test Accuracy: {knn.score(X_test, y_test)}")
print(f"Train Accuracy: {knn.score(X_train, y_train)}")

C:\Users\Jcsst\ana\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Test Accuracy: 0.837964942283027


C:\Users\Jcsst\ana\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Train Accuracy: 0.8705366688047894


# Cross Validation

In [7]:
x, y = ready_data_convert(pd.concat([df_2016, df_2017, df_2018]))
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)

parameters = {
    'n_neighbors': range(1, 20),
}
knn = RandomizedSearchCV(KNeighborsClassifier(), parameters, random_state=0, n_iter=200)

knn.fit(X_train, y_train)

#dump(knn.best_estimator_, '../models/random_forest/convert_cv.joblib')

print(knn.best_params_)
print(f"Test Accuracy: {knn.score(X_test, y_test)}")
print(f"Train Accuracy: {knn.score(X_train, y_train)}")

report = classification_report(y_test, knn.predict(X_test), zero_division=0)
print(report)

C:\Users\Jcsst\ana\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 19 is smaller than n_iter=200. Running 19 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
C:\Users\Jcsst\ana\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\Jcsst\ana\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts alon

{'n_neighbors': 12}


C:\Users\Jcsst\ana\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Test Accuracy: 0.8182984181274049


C:\Users\Jcsst\ana\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Train Accuracy: 0.8362197990164636


C:\Users\Jcsst\ana\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


              precision    recall  f1-score   support

           0       0.12      0.01      0.02        85
   CONVERTED       0.43      0.10      0.16       160
      FAILED       0.41      0.21      0.28       154
  FIELD_GOAL       0.75      0.92      0.83       542
        PUNT       0.88      0.97      0.93      1398

    accuracy                           0.82      2339
   macro avg       0.52      0.45      0.44      2339
weighted avg       0.76      0.82      0.78      2339

